In [1]:
import sys
import os
import time
from geopy.geocoders import Nominatim, GoogleV3, Bing
import folium
from bs4 import BeautifulSoup
import requests
import numpy as np
import googleCreds
import pandas as pd
geolocator = GoogleV3(api_key = googleCreds.GOOGLE_API_KEY)

### 1. Companies' markers as a feature group

In [2]:
dfCompanies = pd.read_csv("dfCompanies.csv")
dfCompanies.dtypes

name        object
lt         float64
lg         float64
address     object
dtype: object

In [3]:
#Creating a dictionary for a future dataset (dfCompanies) and adding markers to the map
feature_group1 = folium.FeatureGroup(name='Companies')

#I need this location to center the future map 
location = geolocator.geocode("Milano Italy")

# map1 = folium.Map(
#     location=[location.latitude, location.longitude],
#     tiles='cartodbpositron',
#     zoom_start=13)
tooltip = 'Click me!'

for index, loc in dfCompanies.iterrows():
#     print([loc["lt"], loc["lg"]])
#     print(f'<i>{loc.name}+": "+{loc.address}</i>')
#     input()
    if "Milano" in set(loc.address.split(" ")) or "MI," in set(loc.address.split(" ")):
        #Adding marker
        folium.Marker([loc["lt"], loc["lg"]], 
                      popup=f'<i>{loc.name}+": "+{loc.address}</i>', 
                      tooltip=tooltip).add_to(feature_group1)
feature_groups = []
feature_groups.append(feature_group1)

### 2. Median and mean location markers as a feature group

In [4]:
meanLat = dfCompanies["lt"].mean()
meanLon = dfCompanies["lg"].mean()

medianLat = dfCompanies["lt"].median()
medianLon = dfCompanies["lg"].median()

medianLocation = (medianLat, medianLon)
meanLocation = (meanLat, meanLon)

In [5]:
feature_group2 = folium.FeatureGroup(name="Optimal Locations based on companies")
folium.Marker([meanLat, meanLon], 
                      popup='<i>Mean location</i>',
                      radius= 50,
                      tooltip=tooltip, 
                      icon=folium.Icon(color='green', 
                                       icon='glyphicon-ok-circle')).add_to(feature_group2)

folium.Marker([medianLat, medianLon], 
                      popup='<i>Median location</i>',
                      radius= 50,
                      tooltip=tooltip, 
                      icon=folium.Icon(color='green', 
                                       icon='glyphicon-ok-circle')).add_to(feature_group2)

#feature_group2.add_to(map1)
feature_groups.append(feature_group2)

### 3. Zones of Milan

In [6]:
milanodistricts = "zonedecentramento.geojson" 
#Source: https://dati.comune.milano.it/dataset/ds153-infogeo-zone-localizzazione-2013
#Last Access Date: 11/08/2020

feature_group3 = folium.FeatureGroup(name='Districts (Zones)')
folium.GeoJson(
    milanodistricts,
    name='geojson').add_to(feature_group3)


feature_groups.append(feature_group3)
# folium.TopoJson(
#     milanodistricts,
#     'objects.antarctic_ice_shelf',
#     name='topojson').add_to(map1)
# folium.LayerControl().add_to(map1)

# map1.choropleth(
#   geo_data=milanodistricts,
#   fill_color='YlOrRd', 
#   fill_opacity=0.7, 
#   line_opacity=0.2
# )
# feature_group3.add_to(map1)
# folium.LayerControl(collapsed = False).add_to(map1)
# map1

### 4. Green Zones

I had to use this map(layer) as a foundation because I couldn't find out how to make choropleth layer toggleable.

In [7]:
dfGreen = pd.read_csv("dfGreen.csv")
dfGreen

,ZONADEC,GreenArea,TotalArea,GreenConc
0,1,1.108470,9.67,0.114630
1,2,1.067198,12.58,0.084833
2,3,1.732050,14.23,0.121718
3,4,2.133899,20.95,0.101857
4,5,1.745979,29.87,0.058453
5,6,2.125662,18.28,0.116283
6,7,5.294581,31.34,0.168940
7,8,3.549903,23.72,0.149659
8,9,3.677692,21.12,0.174133


In [8]:
mapGreen = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=12,
    tiles='cartodbpositron')


choropleth = folium.Choropleth('zonedecentramento.geojson',
                  name='Vegetation concentration',
                  data=dfGreen,
                  key_on="properties.ZONADEC",
                  columns=['ZONADEC', 'GreenConc'],
                  fill_color='YlGn',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name='Vegetation Concentration (green area per total area)',
                  show = False
).add_to(mapGreen)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['ZONADEC'])
)
# feature_group1.add_to(mapGreen)
# feature_group2.add_to(mapGreen)
# feature_group3.add_to(mapGreen)

### 5. Environmental monitoring stations 

In [9]:
dfAir3=pd.read_csv("5_dfAirScaledMedianStations.csv")
dfAir3

,id_amat,valore,normv,name,coords,lt,lg
0,2,12.0,0.153846,via Pascal *,"[45.4740982055664, 9.23478031158447]",45.474098,9.23478
1,3,2.2,0.281250,viale Liguria,"[45.4441986083984, 9.16944026947021]",45.444199,9.16944
2,4,10.0,0.240000,viale Marche,"[45.4962997436523, 9.19083976745605]",45.496300,9.19084
3,6,14.0,0.214953,via Senato *,"[45.4705009460449, 9.19791984558105]",45.470501,9.19792
4,7,52.0,0.261397,Verziere,"[45.4635009765625, 9.19534015655518]",45.463501,9.19534


In [10]:
import json
with open('qaria_stazione.geojson') as data_file:
    data = json.load(data_file)

def color_producer(value, mx, inv = False):
    if inv == False:
        if value < 0.3*mx:
            return 'green'
        elif value < 0.75*mx:
            return 'orange'
        else:
            return 'red'
    else:
        if value < 0.3*mx:
            return 'red'
        elif value < 0.75*mx:
            return 'orange'
        else:
            return 'green'
        
feature_group5 = folium.FeatureGroup(name='Environmental monitoring stations ')
# mapB = folium.Map(
#     location=[location.latitude, location.longitude],
#     zoom_start=13,
#     tiles='cartodbpositron')
mx = max(dfAir3["normv"])

    
for i, row in dfAir3.iterrows(): #data["features"]):
    #print(f["geometry"]["coordinates"])
    try:
        folium.Circle(location = [data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][1],
                                  data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][0]], 
                      radius = 1500,
                      fill = True,
                      fill_color = color_producer(row["normv"], mx),
                      weight = 0,
                      popup = f"Station #{i}. Conc: "+str(round(row["normv"],2))).add_to(feature_group5)
    except:
        folium.Circle(location = [data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][1],
                                  data["features"][int(row["id_amat"])-1]["geometry"]["coordinates"][0]], 
                      radius = 1500,
                      fill = True,
                      fill_color = "grey",
                      weight = 0, 
                      popup = "NA").add_to(feature_group5)

# feature_group5.add_to(mapGreen)
feature_groups.append(feature_group5)

### 6. Predicted Air Quality scores

In [11]:
minmaxlat = [45.39, 45.53]
minmaxlong = [9.08, 9.29]

In [12]:
from folium.plugins import HeatMap


dct = {"lt":[],
      "lg": []}
for i in np.arange(minmaxlat[0], minmaxlat[1], 0.01):
    for j in np.arange(minmaxlong[0], minmaxlong[1], 0.01):
        dct["lt"].append(i)
        dct["lg"].append(j)
        
dfHeatmap = pd.DataFrame(dct)
dfHeatmap = dfHeatmap.append(dfAir3[["lt","lg"]])

In [13]:
import pickle
pkl_filename = "models/normv_lt_lg_LinearRegression.pkl"
with open(pkl_filename, 'rb') as file:
    normv_lt_lg_LinearRegression = pickle.load(file)

In [14]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=3)

In [15]:
dfHeatmap["contamination"] = normv_lt_lg_LinearRegression.predict(pf.fit_transform(dfHeatmap))
dfHeatmap

,lt,lg,contamination
0,45.390000,9.08000,-1.183372
1,45.390000,9.09000,-0.789219
2,45.390000,9.10000,-0.405414
3,45.390000,9.11000,-0.031859
4,45.390000,9.12000,0.331544
...,...,...,...
0,45.474098,9.23478,0.150602
1,45.444199,9.16944,0.281250
2,45.496300,9.19084,0.233645
3,45.470501,9.19792,0.200935


In [16]:
dfHeatmap["contamination"] = dfHeatmap["contamination"].map(lambda x: max(0, x))
dfHeatmap

,lt,lg,contamination
0,45.390000,9.08000,0.000000
1,45.390000,9.09000,0.000000
2,45.390000,9.10000,0.000000
3,45.390000,9.11000,0.000000
4,45.390000,9.12000,0.331544
...,...,...,...
0,45.474098,9.23478,0.150602
1,45.444199,9.16944,0.281250
2,45.496300,9.19084,0.233645
3,45.470501,9.19792,0.200935


In [17]:
import branca.colormap as cm
from collections import defaultdict
steps = 20
#color_map=cm.linear.Blues_03.scale(0,1).to_step(steps)
color_map=cm.linear.YlOrRd_09.scale(0,1).to_step(steps)

feature_group6 = folium.FeatureGroup(name='Air Contamination Heatmap')
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)

# mapB = folium.Map(
#     location=[location.latitude, location.longitude],
#     zoom_start=13,
#     tiles='cartodbpositron')

HeatMap(dfHeatmap, min_opacity = 0.3, 
        radius =50,
        blur = 50,
       gradient = gradient_map).add_to(folium.FeatureGroup(name='Heat Map').add_to(feature_group6))

#feature_group6.add_to(mapGreen)
feature_groups.append(feature_group6)

### 7. Apartments for rent (scored)

In [18]:
minPrice = 1000
maxPrice = 2000

In [19]:
dfObjects=pd.read_csv("dfObjects.csv")
dfObjects = dfObjects[(dfObjects["price"]>=minPrice) & (dfObjects["price"]<=maxPrice)]
dfObjects = dfObjects[dfObjects["type"] == "Bilocale"]
dfObjects.head()

,id,type,address,price,coords,contamination,distanceToDangerZone,distanceToMedianJobLocation,Zone,GreenConc,lt,lg,priceLogScaled,totalScore
4,link_ad_83000949,Bilocale,"via Carlo Imbonati 64, Dergano, Milano",1300,"[45.50395839999999, 9.182326399999999]",0.359669,1.781589,4.291826,9.0,0.174133,45.503958,9.182326,0.377121,-15.364468
12,link_ad_83770907,Bilocale,"viale Legioni Romane 8, Bande Nere, Milano",1050,"[45.46091029999999, 9.1343716]",-0.104809,2.079047,4.348039,6.0,0.116283,45.460910,9.134372,0.292828,-11.703638
13,link_ad_82869489,Bilocale,"corso Magenta, San Vittore, Milano",1500,"[45.4657096, 9.1717594]",0.156972,1.818443,1.399301,1.0,0.114630,45.465710,9.171759,0.433601,-0.283998
14,link_ad_81424435,Bilocale,"via Pinamonte da Vimercate, Moscova, Milano",1250,"[45.4782881, 9.1823255]",0.180052,3.243976,1.512389,1.0,0.114630,45.478288,9.182325,0.361642,6.522617
20,link_ad_83171835,Bilocale,"via Cucchiari 21, Cenisio, Milano",1650,"[45.4900587, 9.1672878]",0.248674,1.920530,3.222590,8.0,0.149659,45.490059,9.167288,0.471218,-9.361462


In [20]:
dfObjects.dtypes

id                              object
type                            object
address                         object
price                            int64
coords                          object
contamination                  float64
distanceToDangerZone           float64
distanceToMedianJobLocation    float64
Zone                           float64
GreenConc                      float64
lt                             float64
lg                             float64
priceLogScaled                 float64
totalScore                     float64
dtype: object

In [21]:
feature_group7 = folium.FeatureGroup(name='Objects (apartments)')


location = geolocator.geocode("Milano Italy")
# map2 = folium.Map(
#     location=[location.latitude, location.longitude],
#     tiles='cartodbpositron',
#     zoom_start=13)
# tooltip = 'Click me!'

for row in dfObjects.iterrows():
    link = "<a href='https://www.immobiliare.it/annunci/"+str(row[1].id.split("_")[2])+"/"+" target='_blank'" + ">Link</a>"
    st = f"{row[1].type},{row[1].id},EUR {row[1].price}"
    folium.Marker([row[1]["lt"], row[1]["lg"]], 
                  popup = st+" "+ link + " Total score:" + str(row[1].totalScore), 
                  tooltip=tooltip,
                  icon=folium.Icon(color=color_producer(row[1].totalScore, max(dfObjects["totalScore"]), 
                                                        inv = True))).add_to(feature_group7)


#feature_group6.add_to(mapGreen)
feature_groups.append(feature_group7)

### 8. Disadvantaged areas

In [22]:
def getLoc(address, annot = False):
    """
    
    """
    try:
        location = geolocator.geocode(address)
        if annot:
            print(address + " was succesfully added\n")
        return [location[1][0], location[1][1]]
        
    except:
        if annot:
            print(address + " was skipped. Probably was not found\n")
        return np.nan

In [23]:
feature_group8 = folium.FeatureGroup(name='Dangerous Zones')
blackList = ["Quarto Oggiaro", "Roserio", "viale Padova", "Bovisa", "Rogored", "Barona", "Corvetto", "San Siro", "Via Gola"]
for item in blackList:
    folium.Circle(getLoc(item),
                  radius = 2000,
                  fill = True,
                  fill_color = "yellow",
                  weight = 0,
                  popup = item, 
                  tooltip=tooltip).add_to(feature_group8)
    
    folium.Circle(getLoc(item),
                  radius = 1000,
                  fill = True,
                  fill_color = "red",
                  weight = 0,
                  popup = item, 
                  tooltip=tooltip).add_to(feature_group8)



#feature_group7.add_to(mapGreen)
feature_groups.append(feature_group8)

# folium.LayerControl(collapsed = False).add_to(mapGreen)
# mapGreen
# mapGreen.save("mapGreen.html") 

### 9. Heat map of predicted total score

In [24]:
dct = {"lt":[],
      "lg": []}
for i in np.arange(minmaxlat[0], minmaxlat[1], 0.01/0.75):
    for j in np.arange(minmaxlong[0], minmaxlong[1], 0.02):
        dct["lt"].append(i)
        dct["lg"].append(j)
        
dfHeatmap = pd.DataFrame(dct)
dfHeatmap = dfHeatmap.append(dfAir3[["lt","lg"]])
dfHeatmap = dfHeatmap.append(dfObjects[["lt","lg"]])
dfHeatmap

,lt,lg
0,45.390000,9.080000
1,45.390000,9.100000
2,45.390000,9.120000
3,45.390000,9.140000
4,45.390000,9.160000
...,...,...
1021,45.481042,9.219500
1023,45.464544,9.142241
1033,45.508549,9.104933
1037,45.512450,9.220291


In [25]:
import pickle
pkl_filename = "models/totalScore_lt_lg_RandomForestRegressor.pkl"
with open(pkl_filename, 'rb') as file:
    totalScore_lt_lg_RandomForestRegressor = pickle.load(file)

In [26]:
dfHeatmap["totalScore"] = totalScore_lt_lg_RandomForestRegressor.predict(dfHeatmap)
dfHeatmap

,lt,lg,totalScore
0,45.390000,9.080000,3.294641
1,45.390000,9.100000,2.031444
2,45.390000,9.120000,1.756827
3,45.390000,9.140000,1.857906
4,45.390000,9.160000,2.142250
...,...,...,...
1021,45.481042,9.219500,1.660030
1023,45.464544,9.142241,3.955276
1033,45.508549,9.104933,3.389168
1037,45.512450,9.220291,0.487858


In [27]:
import branca.colormap as cm
from collections import defaultdict
steps = 50
#color_map=cm.linear.Blues_03.scale(0,1).to_step(steps)
color_map=cm.linear.RdBu_03.scale(0,1).to_step(steps)

feature_group9 = folium.FeatureGroup(name='Predicted Total Score')
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)

# mapB = folium.Map(
#     location=[location.latitude, location.longitude],
#     zoom_start=13,
#     tiles='cartodbpositron')

HeatMap(dfHeatmap, min_opacity = 0.5, 
        #radius =50,
        blur = 30,
       gradient = gradient_map).add_to(folium.FeatureGroup(name='Heat Map').add_to(feature_group9))

#feature_group6.add_to(mapGreen)
feature_groups.append(feature_group9)

### Plotting the map

In [28]:
for f in feature_groups:
    f.add_to(mapGreen)
folium.LayerControl(collapsed = False).add_to(mapGreen)
mapGreen
mapGreen.save("mapGreen.html") 

### 10. Forming an interactive Dash

In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import random
from update import updateMap, updateJob

import os
import glob
import re
import time


In [2]:
ts = time.time()
print(str(int(time.time())))

1606751341


In [3]:
external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

In [4]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [5]:
app.layout = html.Div([
    html.Div([
        html.Div([
            html.H2("Where to live in Milan? | Dove abitare a Milano?"),
#             dcc.Dropdown(id="slct_season",
#                         options = [{'label':'Bilocale', 'value':1},
#                                    {'label':'Trilocale', 'value':2},
#                                    {'label':'Loft', 'value':3},
#                                    {'label':'winter', 'value':4}],
#                         multi = False,
#                         value = 1)],
        html.B("Job name | Nome del lavoro"),
        html.Br(),
        dcc.Input(
            id="job_name",
            type="text",
            debounce = True,
            style = {'width': '80%', 'font-size': '20px'},
            placeholder="Enter job name...",)],
        style={'width': '48%', 
               'display': 'inline-block', 
               'margin': '1em'}),
        
        html.Div([
            html.B("    Price importance | Importanza del prezzo"),
            html.Br(),
            dcc.Slider(min=0,
                       max=10,
                       marks={i: '{}'.format(i) for i in range(11)},
                       value=5, id = "price_importance"),
            
            html.Br(),
            html.B("    Distance from work importance | Importanza della distanza dal lavoro"),
            html.Br(),
            dcc.Slider(min=0,
                       max=10,
                       marks={i: '{}'.format(i) for i in range(11)},
                       value=5, id = "work_importance"),
            
            html.Br(),
            html.B("    Distance from disadvantaged areas importance | Importanza della distanza dalle zone svantaggiate"),
            html.Br(),
            dcc.Slider(min=0,
                       max=10,
                       marks={i: '{}'.format(i) for i in range(11)},
                       value=5, id = "danger_importance"),
            
            html.Br(),
            html.B("    Air quality importance | Importanza della qualità dell'aria"),
            html.Br(),
            dcc.Slider(min=0,
                       max=10,
                       marks={i: '{}'.format(i) for i in range(11)},
                       value=5, id = "air_importance"),
            
            html.Br(),
            html.B("    Green area importance | Importanza dell'area verde"),
            html.Br(),
            dcc.Slider(min=0,
                       max=10,
                       marks={i: '{}'.format(i) for i in range(11)},
                       value=5, id = "green_importance")
        ],
        style={'width': '30%', 'align': 'right', 'display': 'inline-block', 'margin': '1em'})
            ]),
    
    html.Div(id="output_container", children = [html.Iframe(srcDoc = open(glob.glob('updatedMap/*')[0], 'r').read(),
                                                           style = {"border": 0, 
                                                                    "width":"100%", 
                                                                    "height": "800px",
                                                                    "overflow": "auto"})]),
    html.Br()#,
    #dcc.Graph(id='season_chart', figure = {})
])

# app.css.append_css({
#     'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
# })

In [6]:
old_job = "data science"

In [7]:
@app.callback(
    [Output(component_id='output_container', component_property='children')],
    [Input(component_id='job_name', component_property='value'),
     Input(component_id='price_importance', component_property='value'), 
     Input(component_id='work_importance', component_property='value'), 
     Input(component_id='danger_importance', component_property='value'), 
     Input(component_id='air_importance', component_property='value'), 
     Input(component_id='green_importance', component_property='value')]
)
def update_graph(job_name, price_importance, work_importance, danger_importance, air_importance, green_importance):
#     print(option_slctd)
#     print(type(option_slctd))
    
    #container = "The year chosen by user was: {}".format(option_slctd)
    global old_job
    #updateJob(job_name)
    if job_name != old_job:
        updateJob(job_name)
        old_job = job_name
        print(10*"="+"JOB UPDATED"+10*"=")
    num = str(int(time.time()))
    
    files = glob.glob('updatedMap/*')
    for f in files:
        os.remove(f)
        
    updateMap(job_name, price_importance, work_importance, danger_importance, air_importance, green_importance, num)
    fig = [html.Iframe(srcDoc = open(f'updatedMap/mapGreenUpdated{num}.html', 'r').read(),
                                                           style = {"border": 0, 
                                                                    "width":"100%", 
                                                                    "height": "800px",
                                                                    "overflow": "auto"})]
    
    return fig
    #return container, fig

In [8]:
%tb
app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


No traceback available to show.
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Nov/2020 18:49:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2020 18:49:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2020 18:49:07] "GET /_dash-layout HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/opt/anaconda3/lib/python3.7/site-packages/dash/dash.py", line 1076, in dispatch
    response.set_d

127.0.0.1 - - [30/Nov/2020 18:49:07] "POST /_dash-update-component HTTP/1.1" 500 -


Gruppo Gelati srl was succesfully added with address:
Via Guglielmo Marconi, 49, 43058 Sorbolo PR, Italy

Epicura was skipped. Probably was not found

Iper, La grande i was succesfully added with address:
Via Don Luigi Palazzolo, 20, 20149 Milano MI, Italy

MYPETCLINIC was succesfully added with address:
Viale Daniele Ranzoni, 10, 20149 Milano MI, Italy

ISTUM Human Resources was succesfully added with address:
Piazzale della Resistenza, 3, 50018 Scandicci FI, Italy

COOP Lombardia was succesfully added with address:
Via Fratelli Zoia, 3, 20153 Milano MI, Italy

MYPETCLINIC was succesfully added with address:
Viale Daniele Ranzoni, 10, 20149 Milano MI, Italy

COOP Lombardia was succesfully added with address:
Via Fratelli Zoia, 3, 20153 Milano MI, Italy

MAXI ZOO ITALIA SPA was succesfully added with address:
Via Lorenteggio, 246, 20147 Milano MI, Italy

Recipharm was skipped. Probably was not found

Denti Doc srl was succesfully added with address:
Piazzetta Guastalla, 20122 Milano MI

...

127.0.0.1 - - [30/Nov/2020 18:53:17] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [30/Nov/2020 18:53:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2020 18:53:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2020 18:53:54] "POST /_dash-update-component HTTP/1.1" 200 -


ETM TECH was skipped. Probably was not found

Assertion succesful. Creating Dataframe
==========JOB UPDATED==========
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.

127.0.0.1 - - [30/Nov/2020 19:28:31] "POST /_dash-update-component HTTP/1.1" 500 -


Arquati S.r.l. was succesfully added with address:
Via Lorenteggio ang, Via Francesco Primaticcio, 2, 20146 Milano MI, Italy

Comcast Italia Srl was succesfully added with address:
Via Senigallia, 20161 Milano MI, Italy

ITALIA MULTIMEDIA SRL was succesfully added with address:
Viale Giustiniano, 5, 20129 Milano MI, Italy

Xriba Italia Srl was succesfully added with address:
Milan, Metropolitan City of Milan, Italy

INNOVA MCA was succesfully added with address:
Via Angelo Brunetti, 7, 20156 Milano MI, Italy

A.L. SECURITY SERVIZI S.r.l. was succesfully added with address:
Via Vincenzo Bellini, 29, 20095 Cusano Milanino MI, Italy

La Compagnia Holding was succesfully added with address:
Piazza Belgioioso, 2, 20121 Milano MI, Italy

Black Nachos srl was succesfully added with address:
Via Aosta, 23, 20155 Milano MI, Italy

Business Competence S.r.l. was skipped. Probably was not found

Phoenix Network was skipped. Probably was not found

Key-web marketing srl was succesfully added with 

127.0.0.1 - - [30/Nov/2020 19:30:24] "POST /_dash-update-component HTTP/1.1" 200 -


#### Difficulties
* Debug mode of Dash app doesn't work properly with notebooks. I had to turn it off
* Iframe update was problematic becaus html file was stored in cache, and didn't update by callback. I had to implement dynamic naming for newly generated html maps which eliminated the problem.

#### To-do List
- [x] Correct formatting of companies' markers
- [ ] Add job description links to companies' markers
- [ ] Add price range choice for apartments search
- [ ] Add a possibility to choose a type of apartments (bilocale, trilocale, etc.)
- [ ] Create functions for forming each dataset
- - [x] Companies data
- - [ ] Dangerous zones data
- - [ ] Green zones data
- - [ ] Air quality data
- - [ ] Real estate data
- [ ] Update total score heatmap on a callback
- [ ] Map layers should be turned off by default (except objects layer and total score heatmap)
- [ ] Add layer controls as app components.
- - [ ] Selective update of layers for eliminating map rerendering from scratch -> layer creation as functions
- - [ ] Layer update function
- [ ] Add status bar/string